# 🏠 IoT Integration Tutorial

Welcome to the Roommate IoT Integration Tutorial! This guide will help you connect smart devices, sensors, and automation systems with your Roommate assistant to create a truly intelligent living environment.

## 🎯 Learning Objectives

By the end of this tutorial, you'll know how to:
- ✅ Set up ESP32 devices for Roommate integration
- ✅ Connect sensors and actuators
- ✅ Implement smart home automation
- ✅ Create custom IoT workflows
- ✅ Monitor and debug IoT connections
- ✅ Build advanced smart room features

## 📋 Prerequisites

- Basic understanding of Roommate setup (see [Getting Started](getting_started.md))
- ESP32 development board
- Arduino IDE or PlatformIO
- Basic electronics knowledge
- WiFi network for device connectivity

## 🏗️ IoT System Architecture

The Roommate IoT system follows a distributed architecture:

```mermaid
graph TB
    subgraph "🏠 Physical Layer"
        A[🌡️ Temperature Sensors]
        B[💡 Smart Lights]
        C[🔌 Smart Plugs]
        D[📷 Cameras]
        E[🔊 Speakers]
        F[🚪 Door Sensors]
    end
    
    subgraph "🖥️ Device Layer"
        G[ESP32 Hub 1]
        H[ESP32 Hub 2]
        I[ESP32 Hub N]
    end
    
    subgraph "📡 Communication Layer"
        J[WiFi Network]
        K[MQTT Broker]
        L[WebSocket Connection]
    end
    
    subgraph "🖥️ Server Layer"
        M[Roommate Server]
        N[IoT Handler]
        O[Device Registry]
        P[Automation Engine]
    end
    
    subgraph "📱 Application Layer"
        Q[Flutter App]
        R[Web Dashboard]
        S[Voice Interface]
    end
    
    A --> G
    B --> G
    C --> H
    D --> H
    E --> I
    F --> I
    
    G --> J
    H --> J
    I --> J
    
    J --> K
    J --> L
    
    K --> M
    L --> M
    
    M --> N
    M --> O
    M --> P
    
    M --> Q
    M --> R
    M --> S
    
    style A fill:#e1f5fe
    style G fill:#fff3e0
    style M fill:#e8f5e8
    style Q fill:#ffebee
```

## 🔧 ESP32 Setup and Configuration

Let's start by setting up an ESP32 device for Roommate integration.

In [ ]:
# Let's install required Python packages for IoT communication
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")

# Install IoT communication packages
packages = ['paho-mqtt', 'websockets', 'asyncio', 'requests']
for package in packages:
    try:
        if package == 'paho-mqtt':
            import paho.mqtt.client as mqtt
        elif package == 'websockets':
            import websockets
        elif package == 'asyncio':
            import asyncio
        elif package == 'requests':
            import requests
        print(f"✅ {package} already available")
    except ImportError:
        install_package(package)

### 📝 ESP32 Arduino Code Template

Here's the basic ESP32 code template for Roommate integration:

In [ ]:
# This cell shows the Arduino C++ code for ESP32 integration
esp32_code_template = '''
// File: esp32/roommate_iot_hub.ino
#include <WiFi.h>
#include <WebSocketsClient.h>
#include <ArduinoJson.h>
#include <DHT.h>

// WiFi Configuration
const char* ssid = "YOUR_WIFI_SSID";
const char* password = "YOUR_WIFI_PASSWORD";

// Roommate Server Configuration
const char* server_host = "192.168.1.100";  // Your Roommate server IP
const int server_port = 3000;
const String device_id = "esp32-hub-001";
const String api_password = "your-api-password";

// Sensor Configuration
#define DHT_PIN 4
#define DHT_TYPE DHT22
#define LED_PIN 2
#define MOTION_PIN 5
#define LIGHT_SENSOR_PIN A0

// Initialize components
DHT dht(DHT_PIN, DHT_TYPE);
WebSocketsClient webSocket;

// Device state
bool ledState = false;
unsigned long lastSensorRead = 0;
const unsigned long SENSOR_INTERVAL = 30000; // 30 seconds

void setup() {
  Serial.begin(115200);
  
  // Initialize pins
  pinMode(LED_PIN, OUTPUT);
  pinMode(MOTION_PIN, INPUT);
  
  // Initialize sensors
  dht.begin();
  
  // Connect to WiFi
  connectWiFi();
  
  // Connect to Roommate server
  connectWebSocket();
  
  Serial.println("🏠 Roommate IoT Hub initialized!");
}

void loop() {
  webSocket.loop();
  
  // Read sensors periodically
  if (millis() - lastSensorRead > SENSOR_INTERVAL) {
    readAndSendSensorData();
    lastSensorRead = millis();
  }
  
  // Check motion sensor
  checkMotionSensor();
  
  delay(100);
}

void connectWiFi() {
  WiFi.begin(ssid, password);
  Serial.print("Connecting to WiFi");
  
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
  
  Serial.println();
  Serial.print("✅ Connected! IP: ");
  Serial.println(WiFi.localIP());
}

void connectWebSocket() {
  webSocket.begin(server_host, server_port, "/iot/connect");
  webSocket.onEvent(webSocketEvent);
  webSocket.setReconnectInterval(5000);
  
  // Send authentication
  DynamicJsonDocument auth(1024);
  auth["type"] = "auth";
  auth["deviceId"] = device_id;
  auth["apiPassword"] = api_password;
  
  String authString;
  serializeJson(auth, authString);
  webSocket.sendTXT(authString);
}

void webSocketEvent(WStype_t type, uint8_t * payload, size_t length) {
  switch(type) {
    case WStype_DISCONNECTED:
      Serial.println("❌ WebSocket Disconnected");
      break;
      
    case WStype_CONNECTED:
      Serial.printf("✅ WebSocket Connected to: %s\n", payload);
      break;
      
    case WStype_TEXT:
      Serial.printf("📥 Received: %s\n", payload);
      handleCommand((char*)payload);
      break;
      
    default:
      break;
  }
}

void handleCommand(String command) {
  DynamicJsonDocument doc(1024);
  deserializeJson(doc, command);
  
  String action = doc["action"];
  
  if (action == "led_on") {
    digitalWrite(LED_PIN, HIGH);
    ledState = true;
    sendResponse("led_status", "on");
  }
  else if (action == "led_off") {
    digitalWrite(LED_PIN, LOW);
    ledState = false;
    sendResponse("led_status", "off");
  }
  else if (action == "get_status") {
    sendDeviceStatus();
  }
}

void readAndSendSensorData() {
  float temperature = dht.readTemperature();
  float humidity = dht.readHumidity();
  int lightLevel = analogRead(LIGHT_SENSOR_PIN);
  
  if (isnan(temperature) || isnan(humidity)) {
    Serial.println("❌ Failed to read DHT sensor!");
    return;
  }
  
  DynamicJsonDocument sensor(1024);
  sensor["type"] = "sensor_data";
  sensor["deviceId"] = device_id;
  sensor["timestamp"] = millis();
  sensor["data"]["temperature"] = temperature;
  sensor["data"]["humidity"] = humidity;
  sensor["data"]["light"] = lightLevel;
  sensor["data"]["led_state"] = ledState;
  
  String sensorString;
  serializeJson(sensor, sensorString);
  webSocket.sendTXT(sensorString);
  
  Serial.printf("📊 Sent sensor data - Temp: %.1f°C, Humidity: %.1f%%, Light: %d\n", 
                temperature, humidity, lightLevel);
}

void checkMotionSensor() {
  static bool lastMotionState = false;
  bool currentMotion = digitalRead(MOTION_PIN);
  
  if (currentMotion != lastMotionState) {
    lastMotionState = currentMotion;
    
    DynamicJsonDocument motion(512);
    motion["type"] = "motion_event";
    motion["deviceId"] = device_id;
    motion["timestamp"] = millis();
    motion["detected"] = currentMotion;
    
    String motionString;
    serializeJson(motion, motionString);
    webSocket.sendTXT(motionString);
    
    Serial.printf("🚶 Motion %s\n", currentMotion ? "detected" : "stopped");
  }
}

void sendResponse(String type, String value) {
  DynamicJsonDocument response(512);
  response["type"] = "response";
  response["deviceId"] = device_id;
  response["responseType"] = type;
  response["value"] = value;
  response["timestamp"] = millis();
  
  String responseString;
  serializeJson(response, responseString);
  webSocket.sendTXT(responseString);
}

void sendDeviceStatus() {
  DynamicJsonDocument status(1024);
  status["type"] = "device_status";
  status["deviceId"] = device_id;
  status["timestamp"] = millis();
  status["status"]["wifi_rssi"] = WiFi.RSSI();
  status["status"]["free_heap"] = ESP.getFreeHeap();
  status["status"]["uptime"] = millis();
  status["status"]["led_state"] = ledState;
  
  String statusString;
  serializeJson(status, statusString);
  webSocket.sendTXT(statusString);
}
'''

print("🖥️ ESP32 Arduino Code Template:")
print(esp32_code_template)

## 🖥️ Server-Side IoT Integration

Now let's look at the server-side code to handle IoT devices:

In [ ]:
# Server-side IoT handler code
server_iot_code = '''
// File: server/iot_handler.ts
import { WebSocket, WebSocketServer } from 'ws';
import { MongoDBHandler } from '../mongodb/index';

interface IoTDevice {
  id: string;
  name: string;
  type: string;
  status: 'online' | 'offline';
  lastSeen: Date;
  capabilities: string[];
  sensorData?: any;
}

interface IoTMessage {
  type: 'auth' | 'sensor_data' | 'motion_event' | 'response' | 'device_status';
  deviceId: string;
  timestamp?: number;
  data?: any;
  [key: string]: any;
}

export class IoTHandler {
  private wss: WebSocketServer;
  private devices: Map<string, IoTDevice> = new Map();
  private connections: Map<string, WebSocket> = new Map();
  private mongoHandler: MongoDBHandler;
  private apiPassword: string;

  constructor(port: number = 8080, mongoHandler: MongoDBHandler, apiPassword: string) {
    this.mongoHandler = mongoHandler;
    this.apiPassword = apiPassword;
    
    this.wss = new WebSocketServer({ port });
    this.wss.on('connection', this.handleConnection.bind(this));
    
    console.log(`🏠 IoT WebSocket server listening on port ${port}`);
    
    // Cleanup disconnected devices every 5 minutes
    setInterval(() => this.cleanupDevices(), 5 * 60 * 1000);
  }

  private handleConnection(ws: WebSocket, request: any) {
    console.log('📱 New IoT device attempting connection');
    
    let deviceId: string | null = null;
    let authenticated = false;

    ws.on('message', async (data: Buffer) => {
      try {
        const message: IoTMessage = JSON.parse(data.toString());
        
        if (message.type === 'auth') {
          const authResult = this.authenticateDevice(message);
          if (authResult.success) {
            deviceId = message.deviceId;
            authenticated = true;
            this.connections.set(deviceId, ws);
            this.registerDevice(deviceId, message);
            
            ws.send(JSON.stringify({
              type: 'auth_response',
              success: true,
              message: 'Device authenticated successfully'
            }));
            
            console.log(`✅ Device ${deviceId} authenticated and connected`);
          } else {
            ws.send(JSON.stringify({
              type: 'auth_response',
              success: false,
              message: authResult.error
            }));
            ws.close();
          }
        } else if (authenticated && deviceId) {
          await this.handleDeviceMessage(deviceId, message);
        } else {
          ws.send(JSON.stringify({
            type: 'error',
            message: 'Device not authenticated'
          }));
        }
      } catch (error) {
        console.error('❌ Error processing IoT message:', error);
        ws.send(JSON.stringify({
          type: 'error',
          message: 'Invalid message format'
        }));
      }
    });

    ws.on('close', () => {
      if (deviceId) {
        console.log(`📱 Device ${deviceId} disconnected`);
        this.connections.delete(deviceId);
        this.updateDeviceStatus(deviceId, 'offline');
      }
    });

    ws.on('error', (error) => {
      console.error('❌ IoT WebSocket error:', error);
    });
  }

  private authenticateDevice(message: IoTMessage): { success: boolean; error?: string } {
    if (!message.deviceId) {
      return { success: false, error: 'Device ID required' };
    }
    
    if (!message.apiPassword || message.apiPassword !== this.apiPassword) {
      return { success: false, error: 'Invalid API password' };
    }
    
    return { success: true };
  }

  private registerDevice(deviceId: string, authMessage: IoTMessage) {
    const device: IoTDevice = {
      id: deviceId,
      name: authMessage.deviceName || deviceId,
      type: authMessage.deviceType || 'esp32',
      status: 'online',
      lastSeen: new Date(),
      capabilities: authMessage.capabilities || ['sensor', 'actuator']
    };
    
    this.devices.set(deviceId, device);
  }

  private async handleDeviceMessage(deviceId: string, message: IoTMessage) {
    this.updateDeviceLastSeen(deviceId);
    
    switch (message.type) {
      case 'sensor_data':
        await this.handleSensorData(deviceId, message);
        break;
        
      case 'motion_event':
        await this.handleMotionEvent(deviceId, message);
        break;
        
      case 'device_status':
        await this.handleDeviceStatus(deviceId, message);
        break;
        
      default:
        console.log(`📥 Unknown message type from ${deviceId}:`, message.type);
    }
  }

  private async handleSensorData(deviceId: string, message: IoTMessage) {
    const device = this.devices.get(deviceId);
    if (device) {
      device.sensorData = {
        ...message.data,
        timestamp: new Date()
      };
    }
    
    // Store in MongoDB
    try {
      if (this.mongoHandler) {
        // Save sensor data to database
        await this.saveSensorData(deviceId, message.data);
      }
    } catch (error) {
      console.error('❌ Error saving sensor data:', error);
    }
    
    console.log(`📊 Sensor data from ${deviceId}:`, message.data);
    
    // Trigger automations based on sensor data
    this.checkAutomations(deviceId, message.data);
  }

  private async handleMotionEvent(deviceId: string, message: IoTMessage) {
    console.log(`🚶 Motion ${message.detected ? 'detected' : 'stopped'} on ${deviceId}`);
    
    if (message.detected) {
      // Trigger motion-based automations
      await this.triggerMotionAutomation(deviceId);
    }
  }

  private async handleDeviceStatus(deviceId: string, message: IoTMessage) {
    const device = this.devices.get(deviceId);
    if (device) {
      device.status = 'online';
      device.lastSeen = new Date();
    }
    
    console.log(`📊 Device status from ${deviceId}:`, message.status);
  }

  // Public methods for controlling devices
  public sendCommand(deviceId: string, command: any): boolean {
    const ws = this.connections.get(deviceId);
    if (ws && ws.readyState === WebSocket.OPEN) {
      ws.send(JSON.stringify(command));
      console.log(`📤 Sent command to ${deviceId}:`, command);
      return true;
    }
    return false;
  }

  public getDevices(): IoTDevice[] {
    return Array.from(this.devices.values());
  }

  public getDevice(deviceId: string): IoTDevice | undefined {
    return this.devices.get(deviceId);
  }

  private updateDeviceLastSeen(deviceId: string) {
    const device = this.devices.get(deviceId);
    if (device) {
      device.lastSeen = new Date();
      device.status = 'online';
    }
  }

  private updateDeviceStatus(deviceId: string, status: 'online' | 'offline') {
    const device = this.devices.get(deviceId);
    if (device) {
      device.status = status;
    }
  }

  private cleanupDevices() {
    const now = new Date();
    const timeout = 10 * 60 * 1000; // 10 minutes
    
    for (const [deviceId, device] of this.devices.entries()) {
      if (now.getTime() - device.lastSeen.getTime() > timeout) {
        device.status = 'offline';
        console.log(`⚠️ Device ${deviceId} marked as offline (timeout)`);
      }
    }
  }

  private async saveSensorData(deviceId: string, data: any) {
    // This would integrate with your MongoDB handler
    // Implementation depends on your specific schema
  }

  private checkAutomations(deviceId: string, sensorData: any) {
    // Implement automation logic based on sensor data
    // Example: Turn on lights when motion detected and light level is low
    if (sensorData.light < 100 && sensorData.motion) {
      this.sendCommand(deviceId, { action: 'led_on' });
    }
  }

  private async triggerMotionAutomation(deviceId: string) {
    // Example motion-based automation
    const device = this.devices.get(deviceId);
    if (device && device.sensorData && device.sensorData.light < 50) {
      this.sendCommand(deviceId, { action: 'led_on' });
      console.log(`💡 Auto-turned on lights for ${deviceId} due to motion in dark room`);
    }
  }
}
'''

print("🖥️ Server-Side IoT Handler Code:")
print(server_iot_code)

## 💻 Python IoT Client Examples

Let's create Python examples for testing and controlling IoT devices:

In [ ]:
import asyncio
import websockets
import json
import requests
from datetime import datetime
import time

# Configuration
ROOMMATE_SERVER = "ws://localhost:8080"  # WebSocket server for IoT
HTTP_SERVER = "http://localhost:3000"    # HTTP API server
API_PASSWORD = "your-api-password"       # Replace with your API password
DEVICE_ID = "python-client-001"          # Unique device identifier

class RoommateIoTClient:
    def __init__(self, server_url, device_id, api_password):
        self.server_url = server_url
        self.device_id = device_id
        self.api_password = api_password
        self.websocket = None
        self.connected = False
    
    async def connect(self):
        """Connect to Roommate IoT WebSocket server"""
        try:
            self.websocket = await websockets.connect(f"{self.server_url}/iot/connect")
            self.connected = True
            print(f"✅ Connected to {self.server_url}")
            
            # Send authentication
            await self.authenticate()
            
            # Start message handler
            await self.message_handler()
            
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            self.connected = False
    
    async def authenticate(self):
        """Authenticate with the server"""
        auth_message = {
            "type": "auth",
            "deviceId": self.device_id,
            "deviceName": "Python IoT Client",
            "deviceType": "virtual",
            "apiPassword": self.api_password,
            "capabilities": ["sensor", "actuator", "automation"]
        }
        
        await self.websocket.send(json.dumps(auth_message))
        print("🔐 Authentication sent")
    
    async def message_handler(self):
        """Handle incoming messages from server"""
        try:
            async for message in self.websocket:
                data = json.loads(message)
                await self.handle_message(data)
        except websockets.exceptions.ConnectionClosed:
            print("❌ WebSocket connection closed")
            self.connected = False
        except Exception as e:
            print(f"❌ Message handler error: {e}")
    
    async def handle_message(self, data):
        """Process messages from server"""
        message_type = data.get('type')
        
        if message_type == 'auth_response':
            if data.get('success'):
                print("✅ Authentication successful")
            else:
                print(f"❌ Authentication failed: {data.get('message')}")
        
        elif message_type == 'command':
            print(f"📥 Received command: {data}")
            await self.execute_command(data)
        
        else:
            print(f"📥 Received message: {data}")
    
    async def execute_command(self, command):
        """Execute commands from server"""
        action = command.get('action')
        
        if action == 'get_status':
            await self.send_device_status()
        elif action == 'simulate_sensor':
            await self.send_simulated_sensor_data()
        else:
            print(f"❓ Unknown command: {action}")
    
    async def send_sensor_data(self, temperature, humidity, light_level):
        """Send sensor data to server"""
        if not self.connected:
            print("❌ Not connected to server")
            return
        
        sensor_message = {
            "type": "sensor_data",
            "deviceId": self.device_id,
            "timestamp": int(time.time() * 1000),
            "data": {
                "temperature": temperature,
                "humidity": humidity,
                "light": light_level
            }
        }
        
        await self.websocket.send(json.dumps(sensor_message))
        print(f"📊 Sent sensor data - Temp: {temperature}°C, Humidity: {humidity}%, Light: {light_level}")
    
    async def send_motion_event(self, detected):
        """Send motion detection event"""
        if not self.connected:
            print("❌ Not connected to server")
            return
        
        motion_message = {
            "type": "motion_event",
            "deviceId": self.device_id,
            "timestamp": int(time.time() * 1000),
            "detected": detected
        }
        
        await self.websocket.send(json.dumps(motion_message))
        print(f"🚶 Motion {'detected' if detected else 'stopped'}")
    
    async def send_device_status(self):
        """Send device status information"""
        if not self.connected:
            print("❌ Not connected to server")
            return
        
        status_message = {
            "type": "device_status",
            "deviceId": self.device_id,
            "timestamp": int(time.time() * 1000),
            "status": {
                "cpu_usage": 25.5,
                "memory_usage": 67.2,
                "uptime": int(time.time()),
                "version": "1.0.0"
            }
        }
        
        await self.websocket.send(json.dumps(status_message))
        print("📊 Sent device status")
    
    async def send_simulated_sensor_data(self):
        """Send simulated sensor data for testing"""
        import random
        
        temperature = round(random.uniform(18.0, 28.0), 1)
        humidity = round(random.uniform(30.0, 70.0), 1)
        light_level = random.randint(0, 1023)
        
        await self.send_sensor_data(temperature, humidity, light_level)
    
    async def disconnect(self):
        """Disconnect from server"""
        if self.websocket:
            await self.websocket.close()
            self.connected = False
            print("👋 Disconnected from server")

# Create client instance
iot_client = RoommateIoTClient(ROOMMATE_SERVER, DEVICE_ID, API_PASSWORD)
print("🏠 Roommate IoT Client initialized")
print(f"📡 Server: {ROOMMATE_SERVER}")
print(f"🆔 Device ID: {DEVICE_ID}")

## 🧪 Testing IoT Functionality

Let's test the IoT system with simulated devices and data:

In [ ]:
async def test_iot_connection():
    """Test basic IoT connection and authentication"""
    print("🧪 Testing IoT Connection...")
    print("=" * 30)
    
    try:
        # Note: This would normally connect to the actual WebSocket server
        # For demonstration, we'll simulate the process
        
        print("1. ✅ Connecting to WebSocket server...")
        await asyncio.sleep(1)  # Simulate connection time
        
        print("2. ✅ Sending authentication...")
        await asyncio.sleep(0.5)
        
        print("3. ✅ Authentication successful")
        await asyncio.sleep(0.5)
        
        print("4. ✅ Device registered and ready")
        
        return True
        
    except Exception as e:
        print(f"❌ Connection test failed: {e}")
        return False

async def simulate_sensor_readings():
    """Simulate various sensor readings"""
    print("\n📊 Simulating Sensor Readings...")
    print("=" * 35)
    
    import random
    
    # Simulate 10 sensor readings
    for i in range(10):
        temperature = round(random.uniform(20.0, 26.0), 1)
        humidity = round(random.uniform(40.0, 60.0), 1)
        light_level = random.randint(100, 800)
        
        print(f"📊 Reading {i+1}: Temp: {temperature}°C, Humidity: {humidity}%, Light: {light_level}")
        
        # Simulate sending to server
        await asyncio.sleep(0.5)
    
    print("✅ Sensor simulation completed")

async def simulate_motion_events():
    """Simulate motion detection events"""
    print("\n🚶 Simulating Motion Events...")
    print("=" * 30)
    
    motion_sequence = [
        (True, "Person enters room"),
        (False, "No motion for 30 seconds"),
        (True, "Motion detected again"),
        (False, "Person leaves room")
    ]
    
    for detected, description in motion_sequence:
        print(f"🚶 {description} - Motion: {'DETECTED' if detected else 'NONE'}")
        await asyncio.sleep(1)
    
    print("✅ Motion simulation completed")

async def test_device_commands():
    """Test sending commands to devices"""
    print("\n📤 Testing Device Commands...")
    print("=" * 30)
    
    commands = [
        {"action": "led_on", "description": "Turn on LED"},
        {"action": "led_off", "description": "Turn off LED"},
        {"action": "get_status", "description": "Get device status"},
        {"action": "set_brightness", "value": 75, "description": "Set brightness to 75%"}
    ]
    
    for command in commands:
        print(f"📤 Sending: {command['description']}")
        print(f"   Command: {command}")
        await asyncio.sleep(0.5)
        print(f"   ✅ Response: Command executed successfully")
        print()
    
    print("✅ Command testing completed")

# Run all tests
async def run_iot_tests():
    """Run comprehensive IoT tests"""
    print("🧪 Starting Comprehensive IoT Tests")
    print("=" * 40)
    
    # Test connection
    connection_success = await test_iot_connection()
    
    if connection_success:
        # Test sensor data
        await simulate_sensor_readings()
        
        # Test motion events
        await simulate_motion_events()
        
        # Test device commands
        await test_device_commands()
        
        print("\n🎉 All IoT tests completed successfully!")
    else:
        print("❌ Cannot proceed with tests due to connection failure")

# Execute the tests
await run_iot_tests()

## 🏠 Smart Home Automation Examples

Let's explore some practical smart home automation scenarios:

In [ ]:
class SmartHomeAutomation:
    def __init__(self):
        self.devices = {}
        self.automations = []
        self.sensor_data = {}
    
    def add_device(self, device_id, device_type, capabilities):
        """Add a device to the smart home system"""
        self.devices[device_id] = {
            'type': device_type,
            'capabilities': capabilities,
            'status': 'online',
            'state': {}
        }
        print(f"✅ Added device: {device_id} ({device_type})")
    
    def add_automation(self, name, condition, action):
        """Add an automation rule"""
        automation = {
            'name': name,
            'condition': condition,
            'action': action,
            'enabled': True
        }
        self.automations.append(automation)
        print(f"✅ Added automation: {name}")
    
    def update_sensor_data(self, device_id, data):
        """Update sensor data and check automations"""
        self.sensor_data[device_id] = data
        print(f"📊 Updated sensor data for {device_id}: {data}")
        
        # Check and execute automations
        self.check_automations()
    
    def check_automations(self):
        """Check all automation conditions and execute actions"""
        for automation in self.automations:
            if automation['enabled'] and self.evaluate_condition(automation['condition']):
                self.execute_action(automation['action'])
                print(f"🤖 Executed automation: {automation['name']}")
    
    def evaluate_condition(self, condition):
        """Evaluate automation condition"""
        # Simple condition evaluation (in real implementation, this would be more robust)
        condition_type = condition.get('type')
        
        if condition_type == 'sensor_threshold':
            device_id = condition['device_id']
            sensor = condition['sensor']
            operator = condition['operator']
            threshold = condition['threshold']
            
            if device_id in self.sensor_data:
                value = self.sensor_data[device_id].get(sensor)
                if value is not None:
                    if operator == 'less_than':
                        return value < threshold
                    elif operator == 'greater_than':
                        return value > threshold
                    elif operator == 'equals':
                        return value == threshold
        
        elif condition_type == 'motion_detected':
            device_id = condition['device_id']
            if device_id in self.sensor_data:
                return self.sensor_data[device_id].get('motion', False)
        
        elif condition_type == 'time_range':
            from datetime import datetime
            current_hour = datetime.now().hour
            start_hour = condition['start_hour']
            end_hour = condition['end_hour']
            return start_hour <= current_hour <= end_hour
        
        return False
    
    def execute_action(self, action):
        """Execute automation action"""
        action_type = action.get('type')
        
        if action_type == 'device_control':
            device_id = action['device_id']
            command = action['command']
            
            if device_id in self.devices:
                print(f"📤 Sending command '{command}' to {device_id}")
                # In real implementation, this would send the actual command
        
        elif action_type == 'notification':
            message = action['message']
            print(f"🔔 Notification: {message}")
        
        elif action_type == 'scene':
            scene_name = action['scene']
            print(f"🎭 Activating scene: {scene_name}")

# Create smart home automation system
smart_home = SmartHomeAutomation()

print("🏠 Smart Home Automation System")
print("=" * 35)

# Add devices
smart_home.add_device("living_room_hub", "esp32", ["temperature", "humidity", "light", "motion"])
smart_home.add_device("bedroom_lights", "smart_light", ["brightness", "color"])
smart_home.add_device("front_door_sensor", "door_sensor", ["open", "closed"])
smart_home.add_device("smart_thermostat", "thermostat", ["temperature_control"])

print()

# Add automation rules
print("🤖 Adding Automation Rules:")

# Auto-lighting based on motion and light level
smart_home.add_automation(
    "Auto Light Control",
    {
        "type": "sensor_threshold",
        "device_id": "living_room_hub",
        "sensor": "light",
        "operator": "less_than",
        "threshold": 100
    },
    {
        "type": "device_control",
        "device_id": "bedroom_lights",
        "command": "turn_on"
    }
)

# Temperature control
smart_home.add_automation(
    "Temperature Control",
    {
        "type": "sensor_threshold",
        "device_id": "living_room_hub",
        "sensor": "temperature",
        "operator": "greater_than",
        "threshold": 25
    },
    {
        "type": "device_control",
        "device_id": "smart_thermostat",
        "command": "cool_down"
    }
)

# Security notification
smart_home.add_automation(
    "Security Alert",
    {
        "type": "motion_detected",
        "device_id": "living_room_hub"
    },
    {
        "type": "notification",
        "message": "Motion detected in living room"
    }
)

print()

In [ ]:
# Test automation scenarios
print("🧪 Testing Automation Scenarios:")
print("=" * 35)

# Scenario 1: Low light triggers auto-lighting
print("\n📊 Scenario 1: Low light level detected")
smart_home.update_sensor_data("living_room_hub", {
    "temperature": 22.5,
    "humidity": 45.0,
    "light": 50,  # Below threshold of 100
    "motion": False
})

# Scenario 2: High temperature triggers cooling
print("\n📊 Scenario 2: High temperature detected")
smart_home.update_sensor_data("living_room_hub", {
    "temperature": 26.5,  # Above threshold of 25
    "humidity": 55.0,
    "light": 200,
    "motion": False
})

# Scenario 3: Motion detection triggers security alert
print("\n📊 Scenario 3: Motion detected")
smart_home.update_sensor_data("living_room_hub", {
    "temperature": 23.0,
    "humidity": 50.0,
    "light": 150,
    "motion": True  # Motion detected
})

# Scenario 4: Normal conditions - no automations triggered
print("\n📊 Scenario 4: Normal conditions")
smart_home.update_sensor_data("living_room_hub", {
    "temperature": 23.5,
    "humidity": 48.0,
    "light": 300,
    "motion": False
})

print("\n✅ Automation testing completed!")

## 🎯 Advanced IoT Features

Let's explore advanced IoT integration features:

In [ ]:
# Voice control integration with IoT
def voice_to_iot_command(voice_text):
    """
    Convert voice commands to IoT device commands
    """
    voice_text = voice_text.lower()
    
    commands = []
    
    # Light controls
    if "turn on lights" in voice_text or "lights on" in voice_text:
        commands.append({
            "device_id": "bedroom_lights",
            "action": "turn_on",
            "description": "Turning on lights"
        })
    
    elif "turn off lights" in voice_text or "lights off" in voice_text:
        commands.append({
            "device_id": "bedroom_lights",
            "action": "turn_off",
            "description": "Turning off lights"
        })
    
    # Temperature controls
    elif "make it warmer" in voice_text or "increase temperature" in voice_text:
        commands.append({
            "device_id": "smart_thermostat",
            "action": "increase_temp",
            "value": 2,
            "description": "Increasing temperature by 2°C"
        })
    
    elif "make it cooler" in voice_text or "decrease temperature" in voice_text:
        commands.append({
            "device_id": "smart_thermostat",
            "action": "decrease_temp",
            "value": 2,
            "description": "Decreasing temperature by 2°C"
        })
    
    # Scene controls
    elif "good night" in voice_text or "bedtime" in voice_text:
        commands.extend([
            {
                "device_id": "bedroom_lights",
                "action": "turn_off",
                "description": "Turning off all lights"
            },
            {
                "device_id": "smart_thermostat",
                "action": "set_temp",
                "value": 20,
                "description": "Setting temperature to 20°C for sleep"
            }
        ])
    
    elif "good morning" in voice_text or "wake up" in voice_text:
        commands.extend([
            {
                "device_id": "bedroom_lights",
                "action": "turn_on",
                "brightness": 50,
                "description": "Turning on lights at 50% brightness"
            },
            {
                "device_id": "smart_thermostat",
                "action": "set_temp",
                "value": 23,
                "description": "Setting temperature to 23°C for morning"
            }
        ])
    
    # Status requests
    elif "room temperature" in voice_text or "how warm is it" in voice_text:
        commands.append({
            "device_id": "living_room_hub",
            "action": "get_temperature",
            "description": "Getting current room temperature"
        })
    
    return commands

# Test voice commands
print("🗣️ Voice Control Integration:")
print("=" * 30)

voice_commands = [
    "Turn on the lights",
    "Make it warmer in here",
    "Good night",
    "Good morning",
    "What's the room temperature?",
    "Turn off lights"
]

for voice_text in voice_commands:
    print(f"\n🗣️ Voice: '{voice_text}'")
    commands = voice_to_iot_command(voice_text)
    
    if commands:
        for cmd in commands:
            print(f"   📤 {cmd['description']}")
            print(f"      Device: {cmd['device_id']}, Action: {cmd['action']}")
    else:
        print("   ❓ No matching IoT commands found")

## 📊 IoT Data Analytics and Monitoring

Let's create monitoring and analytics for IoT devices:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import random

# Note: In a Jupyter environment, you might need to install matplotlib:
# !pip install matplotlib

class IoTAnalytics:
    def __init__(self):
        self.sensor_history = {}
        self.device_health = {}
        self.automation_logs = []
    
    def add_sensor_reading(self, device_id, timestamp, data):
        """Add sensor reading to history"""
        if device_id not in self.sensor_history:
            self.sensor_history[device_id] = []
        
        self.sensor_history[device_id].append({
            'timestamp': timestamp,
            'data': data
        })
    
    def generate_sample_data(self, device_id, hours=24):
        """Generate sample sensor data for demonstration"""
        base_time = datetime.now() - timedelta(hours=hours)
        
        for i in range(hours * 4):  # Every 15 minutes
            timestamp = base_time + timedelta(minutes=i * 15)
            
            # Simulate realistic sensor data with daily patterns
            hour = timestamp.hour
            
            # Temperature varies with time of day
            base_temp = 20 + 5 * np.sin((hour - 6) * np.pi / 12)
            temperature = base_temp + random.uniform(-2, 2)
            
            # Humidity varies inversely with temperature
            humidity = 70 - (temperature - 20) * 2 + random.uniform(-5, 5)
            
            # Light level follows day/night cycle
            if 6 <= hour <= 18:  # Daytime
                light = 300 + 200 * np.sin((hour - 6) * np.pi / 12) + random.uniform(-50, 50)
            else:  # Nighttime
                light = random.uniform(0, 50)
            
            data = {
                'temperature': round(max(15, min(30, temperature)), 1),
                'humidity': round(max(30, min(80, humidity)), 1),
                'light': max(0, int(light))
            }
            
            self.add_sensor_reading(device_id, timestamp, data)
    
    def plot_sensor_data(self, device_id):
        """Plot sensor data over time"""
        if device_id not in self.sensor_history:
            print(f"❌ No data found for device {device_id}")
            return
        
        history = self.sensor_history[device_id]
        timestamps = [entry['timestamp'] for entry in history]
        temperatures = [entry['data']['temperature'] for entry in history]
        humidity = [entry['data']['humidity'] for entry in history]
        light = [entry['data']['light'] for entry in history]
        
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
        fig.suptitle(f'IoT Sensor Data - {device_id}', fontsize=16)
        
        # Temperature plot
        ax1.plot(timestamps, temperatures, 'r-', linewidth=2, label='Temperature')
        ax1.set_ylabel('Temperature (°C)')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Humidity plot
        ax2.plot(timestamps, humidity, 'b-', linewidth=2, label='Humidity')
        ax2.set_ylabel('Humidity (%)')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # Light plot
        ax3.plot(timestamps, light, 'y-', linewidth=2, label='Light Level')
        ax3.set_ylabel('Light Level')
        ax3.set_xlabel('Time')
        ax3.grid(True, alpha=0.3)
        ax3.legend()
        
        plt.tight_layout()
        plt.xticks(rotation=45)
        plt.show()
    
    def calculate_statistics(self, device_id):
        """Calculate statistics for sensor data"""
        if device_id not in self.sensor_history:
            return None
        
        history = self.sensor_history[device_id]
        temperatures = [entry['data']['temperature'] for entry in history]
        humidity = [entry['data']['humidity'] for entry in history]
        light = [entry['data']['light'] for entry in history]
        
        stats = {
            'temperature': {
                'min': min(temperatures),
                'max': max(temperatures),
                'avg': round(sum(temperatures) / len(temperatures), 1),
                'readings': len(temperatures)
            },
            'humidity': {
                'min': min(humidity),
                'max': max(humidity),
                'avg': round(sum(humidity) / len(humidity), 1),
                'readings': len(humidity)
            },
            'light': {
                'min': min(light),
                'max': max(light),
                'avg': round(sum(light) / len(light), 1),
                'readings': len(light)
            }
        }
        
        return stats

# Create analytics instance
analytics = IoTAnalytics()

print("📊 IoT Analytics System")
print("=" * 25)

# Generate sample data for demonstration
print("📈 Generating sample sensor data...")
analytics.generate_sample_data("living_room_hub", hours=48)
analytics.generate_sample_data("bedroom_hub", hours=24)

print("✅ Sample data generated")

# Calculate and display statistics
devices = ["living_room_hub", "bedroom_hub"]

for device_id in devices:
    print(f"\n📊 Statistics for {device_id}:")
    stats = analytics.calculate_statistics(device_id)
    
    if stats:
        for sensor, data in stats.items():
            print(f"   {sensor.title()}:")
            print(f"     Min: {data['min']}, Max: {data['max']}, Avg: {data['avg']}")
            print(f"     Total readings: {data['readings']}")

print("\n📈 Note: In a real Jupyter environment, you could plot the data using:")
print("analytics.plot_sensor_data('living_room_hub')")

## 🔧 Troubleshooting IoT Issues

Common IoT problems and their solutions:

In [ ]:
def diagnose_iot_system():
    """
    Comprehensive IoT system diagnostics
    """
    print("🔍 IoT System Diagnostics")
    print("=" * 25)
    
    # Simulate diagnostic checks
    diagnostics = {
        "WiFi Connectivity": {"status": "✅ OK", "details": "Signal strength: -45 dBm"},
        "WebSocket Connection": {"status": "✅ OK", "details": "Connected to server"},
        "Device Authentication": {"status": "✅ OK", "details": "API key valid"},
        "Sensor Readings": {"status": "⚠️ WARNING", "details": "Temperature sensor showing irregular readings"},
        "Memory Usage": {"status": "✅ OK", "details": "67% used (within normal range)"},
        "Battery Level": {"status": "⚠️ WARNING", "details": "Battery at 15% - consider charging"},
        "Firmware Version": {"status": "✅ OK", "details": "v1.2.3 (latest)"}
    }
    
    for check, result in diagnostics.items():
        print(f"{result['status']} {check}: {result['details']}")
    
    return diagnostics

def troubleshooting_guide():
    """
    Display troubleshooting guide for common issues
    """
    guide = {
        "Connection Issues": [
            "Check WiFi network status and signal strength",
            "Verify server IP address and port configuration",
            "Ensure firewall allows WebSocket connections",
            "Restart the ESP32 device",
            "Check API password configuration"
        ],
        "Sensor Reading Problems": [
            "Check sensor wiring and connections",
            "Verify sensor power supply voltage",
            "Calibrate sensors if readings seem incorrect",
            "Check for environmental interference",
            "Replace faulty sensors if necessary"
        ],
        "Automation Not Working": [
            "Verify automation rules and conditions",
            "Check device capabilities and commands",
            "Ensure devices are online and responsive",
            "Review automation logs for errors",
            "Test individual device commands manually"
        ],
        "Performance Issues": [
            "Check memory usage and free up resources",
            "Reduce sensor reading frequency if needed",
            "Optimize WiFi network for IoT devices",
            "Update firmware to latest version",
            "Consider device placement for better signal"
        ],
        "Power and Battery Issues": [
            "Monitor battery levels regularly",
            "Implement sleep modes for battery-powered devices",
            "Check charging circuits and power supplies",
            "Optimize code for power efficiency",
            "Consider solar or external power sources"
        ]
    }
    
    print("\n🛠️ IoT Troubleshooting Guide")
    print("=" * 30)
    
    for category, solutions in guide.items():
        print(f"\n❗ {category}:")
        for i, solution in enumerate(solutions, 1):
            print(f"   {i}. {solution}")

def device_health_check():
    """
    Perform health check on IoT devices
    """
    print("\n🏥 Device Health Check")
    print("=" * 22)
    
    devices = {
        "esp32-hub-001": {
            "uptime": "15 days, 3 hours",
            "memory_free": "45%",
            "last_seen": "2 minutes ago",
            "sensor_errors": 0,
            "wifi_signal": "-52 dBm",
            "health": "Excellent"
        },
        "bedroom-lights-002": {
            "uptime": "8 days, 12 hours",
            "memory_free": "78%",
            "last_seen": "30 seconds ago",
            "sensor_errors": 2,
            "wifi_signal": "-67 dBm",
            "health": "Good"
        },
        "door-sensor-003": {
            "uptime": "2 days, 6 hours",
            "memory_free": "23%",
            "last_seen": "5 minutes ago",
            "sensor_errors": 1,
            "wifi_signal": "-78 dBm",
            "health": "Needs Attention"
        }
    }
    
    for device_id, stats in devices.items():
        health_emoji = {
            "Excellent": "🟢",
            "Good": "🟡", 
            "Needs Attention": "🔴"
        }.get(stats["health"], "⚪")
        
        print(f"\n{health_emoji} {device_id}:")
        print(f"   Health: {stats['health']}")
        print(f"   Uptime: {stats['uptime']}")
        print(f"   Memory Free: {stats['memory_free']}")
        print(f"   Last Seen: {stats['last_seen']}")
        print(f"   WiFi Signal: {stats['wifi_signal']}")
        print(f"   Sensor Errors: {stats['sensor_errors']}")

# Run diagnostics
diagnose_iot_system()
troubleshooting_guide()
device_health_check()

## 🎓 Conclusion

Congratulations! You've completed the Roommate IoT Integration Tutorial. Here's what you've learned:

### ✅ Key Takeaways

1. **IoT Architecture**: Understanding the distributed system architecture with ESP32 hubs, WebSocket communication, and server integration

2. **ESP32 Programming**: How to program ESP32 devices for sensor reading, actuator control, and communication with Roommate

3. **Server Integration**: WebSocket-based communication, device authentication, and command handling

4. **Smart Home Automation**: Creating automation rules, condition evaluation, and action execution

5. **Voice Control**: Integrating voice commands with IoT device control

6. **Analytics and Monitoring**: Data collection, visualization, and device health monitoring

7. **Troubleshooting**: Common issues, diagnostic procedures, and solutions

### 🚀 Next Steps

- **Hardware Setup**: Build your first ESP32-based IoT hub with sensors
- **Custom Automations**: Create personalized automation rules for your living space
- **Voice Integration**: Connect with the memory system for contextual voice commands
- **Advanced Sensors**: Integrate cameras, environmental sensors, and actuators
- **Mobile Control**: Use the Flutter app for remote IoT device management

### 📚 Additional Resources

- [Memory System Tutorial](memory_tutorial.ipynb) - Learn how IoT integrates with user memory
- [Architecture Guide](architecture.md) - Deep dive into system architecture
- [ESP32 Documentation](../esp32/README.md) - Hardware-specific setup guides
- [API Reference](api_reference.md) - Complete API documentation
- [Troubleshooting Guide](troubleshooting.md) - Extended troubleshooting information

### 🛠️ Hardware Shopping List

To get started with your IoT setup:

- **ESP32 Development Board** (ESP32-DevKitC or similar)
- **DHT22 Temperature/Humidity Sensor**
- **PIR Motion Sensor**
- **Photoresistor (Light Sensor)**
- **LEDs and Resistors**
- **Breadboard and Jumper Wires**
- **Micro USB Cable**
- **5V Power Supply**

### 🌟 Community and Support

- **GitHub Issues**: Report bugs and request features
- **Discussions**: Share your IoT projects and get help
- **Contributing**: Help improve the IoT integration

---

**Welcome to the future of smart living with Roommate! 🏠🤖🌟**